# Multi-task recommenders


In the [basic retrieval tutorial](https://github.com/tensorflow/recommenders/examples/tfrs_movielens.ipynb) we built a retrieval system using movie watches as positive interaction signals.

In many applications, however, there are multiple rich sources of feedback to draw upon. For example, an e-commerce site may record user visits to product pages (abundant, but relatively low signal), image clicks, adding to cart, and, finally, purchases. It may even record post-purchase signals such as reviews and returns.

Integrating all these different forms of feedback is critical to building systems that users love to use, and that do not optimize for any one metric at the expense of overall performance.

In this tutorial, we are going to build a multi-objective recommender for Movielens, using both implicit (movie watches) and explicit signals (ratings).

## Imports


Let's first get our imports out of the way.


In [0]:
import os
import pprint
import tempfile

from typing import Dict, Text

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

In [0]:
import tensorflow_recommenders as tfrs

## Preparing the dataset

We're going to use the Movielens 100K dataset.

In [0]:
ratings = tfds.load('movie_lens/100k-ratings', split="train")
movies = tfds.load('movie_lens/100k-movies', split="train")

# Select the basic features.
ratings = ratings.map(lambda x: {
    "movie_id": x["movie_id"],
    "user_id": x["user_id"],
    "user_rating": x["user_rating"],
})
movies = movies.map(lambda x: {
    "movie_id": x["movie_id"],
})

And repeat our preparations for building vocabularies and splitting the data into a train and a test set:

In [0]:
# Randomly shuffle data and split between train and test.
tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

movie_ids = movies.batch(100_000).map(lambda x: x["movie_id"])
user_ids = ratings.batch(100_000).map(lambda x: x["user_id"])

unique_movie_ids = np.unique(np.concatenate(list(movie_ids)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

## A multi-task model

There are two critical parts to multi-task recommenders:

1. They optimize for two or more objectives, and so have two or more losses.
2. They share variables between the tasks, allowing for transfer learning.

In this tutorial, we will define our models as before, but instead of having  a single task, we will have two tasks: one that predicts ratings, and one that predicts movie watches.

The user and movie models are as before:

In [0]:
# We wrap these in functions this time so that we can run multiple independent
# models later.
class UserModel(tf.keras.Model):

  def __init__(self, embedding_dimension=32):
    super(UserModel, self).__init__()
    # The model itself is a single embedding layer.
    # However, we could expand this to an arbitrarily complicated Keras model, as long
    # as the output is an vector `embedding_dimension` wide.
    user_features = [tf.feature_column.embedding_column(
        tf.feature_column.categorical_column_with_vocabulary_list(
            "user_id", unique_user_ids,
        ),
        embedding_dimension,
    )]
    self.embedding_layer = tf.keras.layers.DenseFeatures(user_features, name="user_embedding")

  def call(self, inputs):
    return self.embedding_layer(inputs)

class MovieModel(tf.keras.Model):

  def __init__(self, embedding_dimension=32):
    super(MovieModel, self).__init__()
    movie_features = [tf.feature_column.embedding_column(
        tf.feature_column.categorical_column_with_vocabulary_list(
            "movie_id", unique_movie_ids,
        ),
        embedding_dimension,
    )]
    self.embedding_layer = tf.keras.layers.DenseFeatures(movie_features, name="movie_embedding")
  
  def call(self, inputs):
    return self.embedding_layer(inputs)

However, now we will have two tasks. The first is the rating task:

In [0]:
tfrs.tasks.RankingTask(
    loss=tf.keras.losses.MeanSquaredError(),
    metrics=[tf.keras.metrics.MeanSquaredError()],
)

Its goal is to predict the ratings as accurately as possible.

The second is the retrieval task:

In [0]:
tfrs.tasks.RetrievalTask(
    corpus_metrics=tfrs.metrics.FactorizedTopK(
        candidates=movies.batch(128)
    )
)

As before, this task's goal is to predict which movies the user will or will not watch.

### Putting it together

We put it all together in a model class.

The new component here is that - since we have two tasks and two losses - we need to decide on how important each loss is. We can do this by giving each of the losses a weight, and treating these weights as hyperparameters. If we assign a large loss weight to the rating task, our model is going to focus on predicting ratings (but still use some information from the retrieval task); if we assign a large loss weight to the retrieval task, it will focus on retrieval instead.

In [0]:
class MovielensModel(tfrs.models.Model):

  def __init__(self, rating_weight: float, retrieval_weight: float) -> None:
    # We take the loss weights in the constructor: this allows us to instantiate
    # several model objects with different loss weights.

    super().__init__()

    # Embeddings as before.
    self.movie_model: tf.keras.layers.Layer = MovieModel()
    self.user_model: tf.keras.layers.Layer = UserModel()

    # A small model to take in user and movie embeddings and predict ratings.
    # We can make this as complicated as we want as long as we output a scalar
    # as our prediction.
    self.rating_model = tf.keras.Sequential([
        tf.keras.layers.Dense(256, activation="relu"),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dense(1),
    ])

    # The tasks.
    self.rating_task: tf.keras.layers.Layer = tfrs.tasks.RankingTask(
        loss=tf.keras.losses.MeanSquaredError(),
        metrics=[tf.keras.metrics.MeanSquaredError()],
    )
    self.retrieval_task: tf.keras.layers.Layer = tfrs.tasks.RetrievalTask(
        corpus_metrics=tfrs.metrics.FactorizedTopK(
            candidates=movies.batch(128).map(self.movie_model)
        )
    )

    # The loss weights.
    self.rating_weight = rating_weight
    self.retrieval_weight = retrieval_weight

  def call(self, features: Dict[Text, tf.Tensor]) -> tf.Tensor:
    # We pick out the user features and pass them into the user model.
    user_embeddings = self.user_model({"user_id": features["user_id"]})
    # And pick out the movie features and pass them into the movie model,
    # getting embeddings back.
    movie_embeddings = self.movie_model(
        {"movie_id": features["movie_id"]}
    )
    
    return (
        user_embeddings,
        movie_embeddings,
        # We apply the multi-layered rating model to a concatentation of
        # user and movie embeddings.
        self.rating_model(
            tf.concat([user_embeddings, movie_embeddings], axis=1)
        ),
    )

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:

    user_embeddings, movie_embeddings, rating_predictions = self(features)

    # We compute the loss for each task.
    rating_loss = self.rating_task(
        labels=features["user_rating"],
        predictions=rating_predictions,
    )
    retrieval_loss = self.retrieval_task(user_embeddings, movie_embeddings)

    # And combine them using the loss weights.
    return (self.rating_weight * rating_loss
            + self.retrieval_weight * retrieval_loss)

### Rating-specialized model

Depending on the weights we assign, the model will encode a different balance of the tasks. Let's start with a model that only considers ratings.

In [0]:
model = MovielensModel(rating_weight=1.0, retrieval_weight=0.0)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:29: UserWarning: Creating resources inside a function passed to Dataset.map() is not supported. Create each resource outside the function, and capture it inside the function to use it.


In [0]:
cached_train = train.shuffle(100_000).batch(8192).cache()
cached_test = test.batch(4096).cache()

In [0]:
model.fit(cached_train, epochs=3)
model.evaluate(cached_test, return_dict=True)

Epoch 1/3
10/10 [==============================] - 22s 2s/step - mean_squared_error: 7.7681 - factorized_top_k_7: 0.0189 - factorized_top_k_7/top_1_categorical_accuracy: 2.1250e-04 - factorized_top_k_7/top_5_categorical_accuracy: 0.0023 - factorized_top_k_7/top_10_categorical_accuracy: 0.0053 - factorized_top_k_7/top_50_categorical_accuracy: 0.0285 - factorized_top_k_7/top_100_categorical_accuracy: 0.0581 - loss: 7.1756
Epoch 2/3
10/10 [==============================] - 21s 2s/step - mean_squared_error: 1.2631 - factorized_top_k_7: 0.0189 - factorized_top_k_7/top_1_categorical_accuracy: 2.1250e-04 - factorized_top_k_7/top_5_categorical_accuracy: 0.0024 - factorized_top_k_7/top_10_categorical_accuracy: 0.0054 - factorized_top_k_7/top_50_categorical_accuracy: 0.0284 - factorized_top_k_7/top_100_categorical_accuracy: 0.0583 - loss: 1.2576
Epoch 3/3
5/5 [==============================] - 3s 638ms/step - mean_squared_error: 1.2281 - factorized_top_k_7: 0.0190 - factorized_top_k_7/top_1_cate

{'factorized_top_k_7': array([0.00025, 0.0026 , 0.00625, 0.0281 , 0.0578 ], dtype=float32),
 'factorized_top_k_7/top_100_categorical_accuracy': 0.05779999867081642,
 'factorized_top_k_7/top_10_categorical_accuracy': 0.0062500000931322575,
 'factorized_top_k_7/top_1_categorical_accuracy': 0.0002500000118743628,
 'factorized_top_k_7/top_50_categorical_accuracy': 0.02810000069439411,
 'factorized_top_k_7/top_5_categorical_accuracy': 0.0026000000070780516,
 'loss': 1.2009674310684204,
 'mean_squared_error': 1.2281196117401123}

The model does OK on predicting ratings (with an MSE of around 1.23), but performs poorly at predicting which movies will be watched or not: its accuracy at 100 is almost 4 times worse than a model trained solely to predict watches.

### Retrieval-specialized model

Let's now try a model that focuses on retrieval only.

In [0]:
model = MovielensModel(rating_weight=0.0, retrieval_weight=1.0)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:29: UserWarning: Creating resources inside a function passed to Dataset.map() is not supported. Create each resource outside the function, and capture it inside the function to use it.


In [0]:
model.fit(cached_train, epochs=3)
model.evaluate(cached_test, return_dict=True)

Epoch 1/3
10/10 [==============================] - 21s 2s/step - mean_squared_error: 13.9885 - factorized_top_k_8: 0.0249 - factorized_top_k_8/top_1_categorical_accuracy: 1.5000e-04 - factorized_top_k_8/top_5_categorical_accuracy: 0.0026 - factorized_top_k_8/top_10_categorical_accuracy: 0.0063 - factorized_top_k_8/top_50_categorical_accuracy: 0.0382 - factorized_top_k_8/top_100_categorical_accuracy: 0.0773 - loss: 70229.4489
Epoch 2/3
10/10 [==============================] - 22s 2s/step - mean_squared_error: 14.0427 - factorized_top_k_8: 0.0981 - factorized_top_k_8/top_1_categorical_accuracy: 0.0012 - factorized_top_k_8/top_5_categorical_accuracy: 0.0156 - factorized_top_k_8/top_10_categorical_accuracy: 0.0340 - factorized_top_k_8/top_50_categorical_accuracy: 0.1604 - factorized_top_k_8/top_100_categorical_accuracy: 0.2791 - loss: 67823.9311
Epoch 3/3
5/5 [==============================] - 4s 729ms/step - mean_squared_error: 14.1269 - factorized_top_k_8: 0.0640 - factorized_top_k_8/top

{'factorized_top_k_8': array([0.0005 , 0.00535, 0.0144 , 0.099  , 0.2007 ], dtype=float32),
 'factorized_top_k_8/top_100_categorical_accuracy': 0.20069999992847443,
 'factorized_top_k_8/top_10_categorical_accuracy': 0.014399999752640724,
 'factorized_top_k_8/top_1_categorical_accuracy': 0.0005000000237487257,
 'factorized_top_k_8/top_50_categorical_accuracy': 0.0989999994635582,
 'factorized_top_k_8/top_5_categorical_accuracy': 0.005350000225007534,
 'loss': 28544.43359375,
 'mean_squared_error': 14.126886367797852}

We get the opposite result: a model that does well on retrieval, but poorly on predicting ratings.

### Equally-weighted model

Let's now train a model that weighs both tasks equally.

In [0]:
model = MovielensModel(rating_weight=1.0, retrieval_weight=1.0)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:29: UserWarning: Creating resources inside a function passed to Dataset.map() is not supported. Create each resource outside the function, and capture it inside the function to use it.


In [0]:
model.fit(cached_train, epochs=3)
model.evaluate(cached_test, return_dict=True)

Epoch 1/3
10/10 [==============================] - 22s 2s/step - mean_squared_error: 7.2046 - factorized_top_k_9: 0.0270 - factorized_top_k_9/top_1_categorical_accuracy: 2.6250e-04 - factorized_top_k_9/top_5_categorical_accuracy: 0.0032 - factorized_top_k_9/top_10_categorical_accuracy: 0.0070 - factorized_top_k_9/top_50_categorical_accuracy: 0.0413 - factorized_top_k_9/top_100_categorical_accuracy: 0.0833 - loss: 70187.2571
Epoch 2/3
10/10 [==============================] - 21s 2s/step - mean_squared_error: 1.3413 - factorized_top_k_9: 0.0999 - factorized_top_k_9/top_1_categorical_accuracy: 0.0011 - factorized_top_k_9/top_5_categorical_accuracy: 0.0162 - factorized_top_k_9/top_10_categorical_accuracy: 0.0351 - factorized_top_k_9/top_50_categorical_accuracy: 0.1629 - factorized_top_k_9/top_100_categorical_accuracy: 0.2840 - loss: 67747.3771
Epoch 3/3
5/5 [==============================] - 4s 732ms/step - mean_squared_error: 1.2156 - factorized_top_k_9: 0.0632 - factorized_top_k_9/top_1_

{'factorized_top_k_9': array([0.00045, 0.00505, 0.0118 , 0.0984 , 0.2005 ], dtype=float32),
 'factorized_top_k_9/top_100_categorical_accuracy': 0.2004999965429306,
 'factorized_top_k_9/top_10_categorical_accuracy': 0.011800000444054604,
 'factorized_top_k_9/top_1_categorical_accuracy': 0.00044999999227002263,
 'factorized_top_k_9/top_50_categorical_accuracy': 0.09839999675750732,
 'factorized_top_k_9/top_5_categorical_accuracy': 0.005049999803304672,
 'loss': 28558.087890625,
 'mean_squared_error': 1.2155885696411133}

The result is a model that performs almost as well as the retrieval-specialized model on retrieval tasks and performs slightly better than the rating-specialized model on rating tasks. This is the promise of transfer learning, where jointly training a model on a set of related tasks can result in a better model that training a set of separate estimators.

Of course, this is not true everywhere. We can expect better results when tasks are closely related, or when we can trasnfer knowledge from a data-abundant task (such as clicks) to a related data-sparse task (such as purchases).